In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# changes directory to the datasets folder
os.chdir("./datasets")
# stores directory to datasets folder to be used later to get the csv files
root = os.getcwd()
# stores files names in current directory (datasets) in a list to be used later get a dataFrame 
files = os.listdir()
# removes hidden file 
files.remove(".DS_Store")
# assign a dictionary to referenced in the future to store dataFrames
datasets = dict()


for item in files:

    # converts csv files to dataFrames
    path = str(f'{root}/{item}')
    df = pd.read_csv(path)

    # gets specific file 
    br = item.split("_")
    targets = br[1:]

    # converts file name from snake case to camel case
    for i in range(len(targets)):
        if i > 0:
            text = targets[i]
            text = text.capitalize()
            targets[i] = text
    target = "".join(targets)
    
    # removes file type from the end of string
    target = target[:-4]

    #stores dataFrames with camel case file name as key
    datasets[target] = df

    # print(targets,target)

# ! there will less datasets when compared to files in dataset folder since the is one instance of datasets spanning multiple questions
# * { files: 30 , datasets: 25}

# changes directory to back to parent directory
os.chdir("../")


In [3]:
logic_df = pd.read_csv('data_.csv')

In [4]:
from Questions import  Question

# stores nested dictionary 
datasets_by_company = dict()

# sorts dataframe by companies
logic_by_company = logic_df.groupby("company_name")

#creates a list of unique company names
companies = list(logic_df["company_name"].unique())

#creates a list of unique question IDs
question_IDs = list(logic_df["question_id"].unique())

#creates a dictionary within another dictionary for referencing questions - company
for company in companies:
    if company not in datasets_by_company.keys():
        datasets_by_company[company] = dict()

#creates a dictionary within another dictionary for referencing questions - difficulty
for company in companies:
    difficulties = list(logic_df[logic_df.company_name == company]["python_difficulty"].unique())
    for difficulty in difficulties:
        datasets_by_company[company][difficulty] = dict()

#creates object for each question and inputs them into dictionary
for question_id in question_IDs:
    question_id_df = logic_df[logic_df['question_id'] == question_id]
    title = question_id_df['title'].unique()[0]
    company_name = question_id_df['company_name'].unique()[0]
    prompt = question_id_df['prompt'].unique()[0]
    question_type = question_id_df['question_type'].unique()[0]
    python_difficulty =  question_id_df['python_difficulty'].unique()[0]
    sql_difficulty =  question_id_df['mySQL_difficulty'].unique()[0]
    filesNames = list(question_id_df['file'].unique())

    f = title.split()
    f = map(str.capitalize,f)
    f = "".join(f)
    
    question = Question(question_id,title,prompt,question_type,python_difficulty,sql_difficulty,filesNames,company_name)

    datasets_by_company[company_name][python_difficulty][f] = question

In [5]:
# Creates a DataFrame of dictionary
test = dict()
for k in datasets_by_company.keys():
    for k2 in datasets_by_company[k].keys():
        for k3 in datasets_by_company[k][k2].keys():
            x = datasets_by_company[k][k2][k3]
            test[k3] = vars(x)

data = pd.DataFrame(test).T

# data

In [6]:
companies.sort()

# filter down by company
company_answer = input(f"Enter company name from following \n {companies}")
# company_answer = "Meta/Facebook"

difficulty_from_company = datasets_by_company[company_answer]


In [7]:
difficulty_from_company = list(difficulty_from_company.keys())

#filter down by difficult
difficult_answer = input(f"Enter difficulty of question \n {difficulty_from_company}")
# difficult_answer = 'medium'

question_from_company = datasets_by_company[company_answer][difficult_answer]

In [8]:
question_from_company = list(question_from_company)

#filter down by question
question_title = input(f"Enter question title from follow \n {question_from_company}")
# question_title = "LikedPosts"

question = datasets_by_company[company_answer][difficult_answer][question_title]


In [9]:

# shows questions info
print(f"Title: {question.title}")
print(f"Prompt: {question.prompt}")
print(f"Python Difficulty: {question.python_difficulty}")
print(f"SQL Difficulty: {question.sql_difficulty}")

Title: Total Order Per Status Per Service
Prompt: Uber is interested in identifying gaps in their business. Calculate the count of orders for each status of each service. Your output should include the service name, status of the order, and the number of orders.
Python Difficulty: easy
SQL Difficulty: easy


In [10]:
from Functions import getDatasetAsDataFrame

#number of rows shown
r = 5

# creates empty df for declaration 
df1 = pd.DataFrame({'A': []})
df2 = pd.DataFrame({'A': []})


# updates the needed df(s) with data
for f in question.fileNames:
    data = getDatasetAsDataFrame(f)
    if df1.empty:
        df1 = data
    elif df2.empty:
        df2 = data

# establish variables for the information of the needed df(s)
data1 = df1.head(r)
column_data1 = df1.dtypes
if df2.empty:
    data2 = ''
    column_data2 = ''
else:
    data2 = df2.head(r)
    column_data2 = df2.dtypes

print(f"Starting DataFrame(s):\n{data1}\n\n{data2}")


Starting DataFrame(s):
  order_date  number_of_orders status_of_order  monetary_value service_name
0   1/1/2016               180       Cancelled      76076000.0     Uber_BOX
1   1/1/2016                10       Cancelled       1960140.0   Uber_CLEAN
2   1/1/2016             28380       Cancelled     489898500.0    Uber_FOOD
3   1/1/2016                 0       Cancelled        728000.0    Uber_GLAM
4   1/1/2016                10       Cancelled        136500.0   Uber_KILAT




In [11]:
print(f"DataFrame Info:\n{column_data1}\n\n{column_data2}")


DataFrame Info:
order_date           object
number_of_orders      int64
status_of_order      object
monetary_value      float64
service_name         object
dtype: object




In [12]:
from Functions import getFinalResult
#get final result
finalResult = getFinalResult(question_title)

q1 = input("Would you like to see final result? (Y/N)")
q1 = str.capitalize(q1)
if(q1 == "Y"):
    print(f"Final Result:\n\n{finalResult}")


Final Result:

    service_name  status_of_order  number_of_orders
0       Uber_BOX        Cancelled              3140
1       Uber_BOX        Completed              6580
2       Uber_BOX  No Driver Found               390
3     Uber_CLEAN        Cancelled               190
4     Uber_CLEAN        Completed              1630
5     Uber_CLEAN  No Driver Found                60
6     Uber_CLEAN            Other                40
7      Uber_FOOD        Cancelled            221420
8      Uber_FOOD        Completed            721210
9      Uber_FOOD  No Driver Found             10750
10     Uber_FOOD            Other              1250
11     Uber_GLAM        Cancelled               130
12     Uber_GLAM        Completed               820
13     Uber_GLAM  No Driver Found               190
14     Uber_GLAM            Other                10
15    Uber_KILAT        Cancelled               130
16    Uber_KILAT        Completed              2200
17    Uber_KILAT  No Driver Found               1

In [13]:
from Functions import getPythonSolution, getSQLSolution, getSolutions

#get solution(s) from directory for question
pythonSolution = getPythonSolution(question_title)
sqlSolution = getSQLSolution(question_title)
solutions = getSolutions(question_title)


q2a = input("Would you like to see solution? (Y/N)")
q2a = str.capitalize(q1)
if(q2a == "Y"):
    q2b = input("Select the solution(s) you will like to see (Python, SQL, Both)")
    if(q2b == "Python"):
        print(pythonSolution)
    elif(q2b == "SQL"):
        print(sqlSolution)
    elif(q2b == "Both"):
        print(solutions)


Python Solution:
# Import your libraries
import pandas as pd

# Start writing code
uber_orders.head()
df = uber_orders.groupby(['service_name', 'status_of_order']).sum()
df = df['number_of_orders'].reset_index()
df 

SQL Solution:
select service_name, status_of_order, sum(number_of_orders) as 'number of orders' 
from uber_orders
group by service_name, status_of_order;

